In [1]:
import numpy as np
import pandas as pd

In [2]:
books = pd.read_csv('Books.csv')
users = pd.read_csv('Users.csv')
ratings = pd.read_csv('Ratings.csv')

C:\Users\malco\AppData\Local\Temp\ipykernel_3520\1748781244.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [3]:
books['Image-URL-M'][1]

'http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg'

In [4]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271379, 8)
(1151856, 3)
(278878, 3)


In [7]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [8]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [9]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [10]:
books.duplicated().sum()

0

In [11]:
ratings.duplicated().sum()

146

In [12]:
users.duplicated().sum()

0

## Popularity Based Recommender System

In [13]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [14]:
ratings_with_name['Book-Rating'] = pd.to_numeric(ratings_with_name['Book-Rating'], errors='coerce')

In [15]:
ratings_with_name.dropna(subset=['Book-Rating'], inplace=True)

In [16]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241079,Ã?Â?lpiraten.,2
241080,Ã?Â?rger mit Produkt X. Roman.,4
241081,Ã?Â?sterlich leben.,1
241082,Ã?Â?stlich der Berge.,3


In [17]:
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating_df

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241079,Ã?Â?lpiraten.,0.000000
241080,Ã?Â?rger mit Produkt X. Roman.,5.250000
241081,Ã?Â?sterlich leben.,7.000000
241082,Ã?Â?stlich der Berge.,2.666667


In [18]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241079,Ã?Â?lpiraten.,2,0.000000
241080,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241081,Ã?Â?sterlich leben.,1,7.000000
241082,Ã?Â?stlich der Berge.,3,2.666667


In [19]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)

In [20]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

In [21]:
popular_df['Image-URL-M'][0]

'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'

## Collaborative Filtering Based Recommender System

In [22]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
padhe_likhe_users = x[x].index

In [23]:
print("User IDs who have rated more than 100 books:")
for user_id in x[x].index:
    print(user_id)

User IDs who have rated more than 100 books:
254
2276
2766
2977
3363
4017
4385
6251
6323
6543
6563
6575
7158
7286
7346
7915
8067
8245
8681
8890
8936
10447
10819
11601
11676
11993
12538
12824
12982
13082
13273
13552
13850
14422
14521
15408
15418
16106
16634
16795
16916
16966
17950
18067
18401
19085
19664
20115
20201
21014
21252
21576
21659
22625
23288
23768
23872
23902
24194
24921
25409
25601
25981
26516
26535
26544
26583
26593
27617
28204
28360
28523
28591
29259
29855
30276
30487
30511
30533
30711
30735
30972
31315
31391
31556
31826
31846
32195
32440
32721
32773
33974
35050
35836
35857
35859
36606
36609
36836
36907
37567
37712
37950
38023
38273
38781
39281
39467
39616
39646
39773
40889
40943
41084
41841
42914
43246
43806
43842
44595
44728
46398
47316
48494
49109
49460
49889
50225
51094
51350
51386
51450
51883
52199
52584
52614
52853
52917
53174
53628
53729
54218
55187
55490
55492
55548
55734
55892
56271
56399
56447
56856
56959
59172
59971
60244
60277
60337
60707
61619
62272
62891
62895

In [24]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [25]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=20
famous_books = y[y].index

In [26]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [27]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [28]:
pt.fillna(0,inplace=True)

In [29]:
# Print filtered books for recommendation
print("Filtered Books for Recommendation:")
for book_title in final_ratings['Book-Title'].unique():
    print(book_title)

Filtered Books for Recommendation:
The Notebook
A Painted House
Lightning
Manhattan Hunt Club
Dark Paradise
Night Sins
Little Altars Everywhere
How Stella Got Her Groove Back
Waiting to Exhale
The Girl Who Loved Tom Gordon : A Novel
The Pillars of the Earth
See Jane Run
The Dark Half
Harry Potter and the Order of the Phoenix (Book 5)
False Memory
Bridget Jones's Diary
Whirlwind (The X-Files)
The X-Files: Ground Zero
Speaking in Tongues
Walk Two Moons
A Wrinkle In Time
The Boy Next Door
Artemis Fowl (Artemis Fowl, Book 1)
Move to Strike
The Last Time They Met : A Novel
Back When We Were Grownups : A Novel (Ballantine Reader's Circle)
The Second Summer of the Sisterhood
Along Came a Spider (Alex Cross Novels)
Alaska
Attack Of The Deranged Mutant Killer Snow Goons
Calvin and Hobbes
A Kiss of Shadows (Meredith Gentry Novels (Paperback))
One
Whispers
Ender's Game (Ender Wiggins Saga (Paperback))
The Jester
The King of Torts
Politically Correct Bedtime Stories: Modern Tales for Our Life and 

In [30]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zlata's Diary: A Child's Life in Sarajevo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zoya,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity_scores = cosine_similarity(pt)

In [33]:
similarity_scores.shape

(3097, 3097)

In [34]:
if 'Think link a monk' in pt.index:
    index = np.where(pt.index == 'Think and Grow Rich')[0]
    if len(index) > 0:
        index = index[0]
        similar_items = sorted(enumerate(similarity_scores[index]), key=lambda x: x[1], reverse=True)[1:5]

        for i, (similar_index, similarity_score) in enumerate(similar_items, start=1):
            similar_book_title = pt.index[similar_index]
            similar_book_details = books[books['Book-Title'] == similar_book_title].drop_duplicates('Book-Title')
            print(f"Recommended Book {i}:")
            print(f"Title: {similar_book_details['Book-Title'].values[0]}")
            print(f"Author: {similar_book_details['Book-Author'].values[0]}")
            print(f"Image URL: {similar_book_details['Image-URL-M'].values[0]}")
            print()
    else:
        print("Book not found in recommendations.")
else:
    print("Book not found in recommendations.")


Book not found in recommendations.


In [35]:
def recommend(book_name):
    book_name = book_name.lower()  # Convert user input to lowercase
    pt_lower = pt.index.str.lower()  # Convert book titles in pt to lowercase

    if book_name in pt_lower:
        index = np.where(pt_lower == book_name)[0]
        if len(index) > 0:
            index = index[0]
            similar_items = sorted(enumerate(similarity_scores[index]), key=lambda x: x[1], reverse=True)[1:5]

            for i, (similar_index, similarity_score) in enumerate(similar_items, start=1):
                similar_book_title = pt.index[similar_index]
                similar_book_details = books[books['Book-Title'] == similar_book_title].drop_duplicates('Book-Title')
                print(f"Recommended Book {i}:")
                print(f"Title: {similar_book_details['Book-Title'].values[0]}")
                print(f"Author: {similar_book_details['Book-Author'].values[0]}")
                print(f"Image URL: {similar_book_details['Image-URL-M'].values[0]}")
                print()
        else:
            print("Book not found in recommendations.")
    else:
        print("Book not found in recommendations.")



In [36]:
recommend('think like a monk')

Recommended Book 1:
Title: Mindset
Author: Carol S. Dweck
Image URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSfy6sl2-JrJDyNb8Ivojzn9qeJwT3Z6pvhYv5_bu-_26sPr3U_pjUSWH0k1P0tCJFTBTA&usqp=CAU

Recommended Book 2:
Title: How to Win Friends and Influence People
Author: Dale Carnegie
Image URL: http://images.amazon.com/images/P/0671027034.01.MZZZZZZZ.jpg

Recommended Book 3:
Title: The Miracle Morning
Author: Hal Elrod
Image URL: https://m.media-amazon.com/images/I/71Eko3myOEL._AC_UY327_FMwebp_QL65_.jpg

Recommended Book 4:
Title: Quiet
Author: Susan Cain
Image URL: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQX62cvKRGHQAB3qF_peWEXxCFu1G19eyDTo-tip7t5fD2YnBQLuA5_iPO9dbhsO-EjSkQ&usqp=CAU



In [37]:
pt.index[545]

'Creed for the Third Millennium'

In [38]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [39]:
books.drop_duplicates('Book-Title')

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271372,6543210987,Start with Why,Simon Sinek,2009,Your Publisher,http://example.com/start_with_why_s.jpg,https://m.media-amazon.com/images/I/41Px2q4eSi...,http://example.com/start_with_why_l.jpg
271373,5432109876,The Miracle Morning,Hal Elrod,2012,Your Publisher,http://example.com/miracle_morning_s.jpg,https://m.media-amazon.com/images/I/71Eko3myOE...,http://example.com/miracle_morning_l.jpg
271375,3210987654,Big Magic,Elizabeth Gilbert,2015,Your Publisher,http://example.com/big_magic_s.jpg,https://m.media-amazon.com/images/I/810KdY8i49...,http://example.com/big_magic_l.jpg
271376,2109876543,Quiet,Susan Cain,2012,Your Publisher,http://example.com/quiet_s.jpg,https://encrypted-tbn0.gstatic.com/images?q=tb...,http://example.com/quiet_l.jpg


In [40]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))